In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import DepReport, Dep, DepKind, ReportDict
from utils import load_pkl
from typing import Dict

report_data: Dict[str, ReportDict] = load_pkl("bcc")

# tool_name = "biotop"
# tool_name = "readahead"

# if tool_name == "biotop":
KEYS = {
    "biotop": [
        DepKind.FUNC("blk_account_io_start"),
        DepKind.FUNC("blk_account_io_done"),
        DepKind.FUNC("__blk_account_io_start"),
        DepKind.FUNC("__blk_account_io_done"),
        DepKind.TRACEPOINT("block_io_start"),
        DepKind.TRACEPOINT("block_io_done"),
        DepKind.FIELD("request::rq_disk"),
        DepKind.FIELD("request_queue::disk"),
    ],
    # ]
    # elif tool_name == "readahead":
    # KEYS = [
    "readahead": [
        DepKind.FUNC("__do_page_cache_readahead"),
        DepKind.FUNC("do_page_cache_ra"),
        DepKind.FUNC("__page_cache_alloc"),
    ],
}

reports: ReportDict = {}

for tool_name, keys in KEYS.items():
    for key in keys:
        reports[key] = report_data[tool_name][key]

[          pkl.py:18 ] INFO: Loding bcc from /Users/szhong/Downloads/bpf-study/output/bcc.pkl


In [50]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.axes import Axes
from matplotlib.patches import Rectangle

from depsurf import VersionGroup
from utils import save_fig

groups = [VersionGroup.REGULAR, VersionGroup.ARCH]

fig, ax = plt.subplots(figsize=(4, 10))
ax: Axes

data = np.array(
    [
        [colors.to_rgb(val.color) for val in report.values()]
        for report in reports.values()
    ]
)


ax.set_xticks(
    np.arange(data.shape[1]),
    labels=[l for g in groups for l in g.labels],
    rotation=90,
    fontsize=8,
)
ax.set_yticks(
    np.arange(data.shape[0]), labels=[d.name for d in reports.keys()], fontsize=7
)
ax.set_xticks(np.arange(data.shape[1] + 1) - 0.5, minor=True)
ax.set_yticks(np.arange(data.shape[0] + 1) - 0.5, minor=True)
ax.imshow(data, aspect=1.2)

ax.grid(which="minor", color="white", linestyle="-", linewidth=0.1)
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
ax.spines["top"].set_visible(True)
ax.spines["bottom"].set_visible(False)
ax.tick_params(which="minor", length=0)


def plot_secondary_ticks(
    ax: Axes,
    axis_name,
    lengths,
    labels,
    pad,
    rotation=0,
    linewidth=0.5,
    linestyle="--",
    fontsize=8,
):
    val = np.array(list(lengths))
    cum = np.cumsum(val)
    mid = cum - val / 2 - 0.5
    div = np.array([0, *cum][:-1]) - 0.5

    if axis_name == "x":
        axis = ax.secondary_xaxis("top").xaxis
    else:
        axis = ax.secondary_yaxis("left").yaxis

    # plot labels
    axis.set_ticks(mid, labels, rotation=rotation, va="center", ha="center", fontsize=fontsize)
    axis.set_tick_params(length=0, pad=pad)

    # plot ticks
    axis.set_ticks(div, labels=[], minor=True)
    axis.set_tick_params(length=pad + 10, width=linewidth, which="minor")

    axline = ax.axvline if axis_name == "x" else ax.axhline
    for d in div:
        axline(d, color="black", lw=linewidth, linestyle=linestyle)


plot_secondary_ticks(
    ax,
    "x",
    [len(g) for g in groups],
    ["Kernel Version", "Arch"],
    linestyle="-",
    pad=40,
)

plot_secondary_ticks(
    ax,
    "y",
    [4, 2, 2, 3],
    ["Func", "Tracept", "Field", "Func"],
    pad=100,
    rotation=90,
)

# plot_secondary_ticks(
#     ax,
#     "y",
#     [8, 3],
#     ["biotop", "read\nahead"],
#     linewidth=1,
#     linestyle="-",
#     pad=130,
#     rotation=90,
# )


for i, report in enumerate(reports.values()):
    for j, val in enumerate(report.values()):
        ax.text(
            j,
            i,
            val.text,
            ha="center",
            va="center",
            color="white",
            fontsize=8,
        )

LEGEND = {
    "OK": "tab:green",
    "Warn": "tab:red",
    "Absent": "lightgray",
}

ax.legend(
    handles=[Rectangle((0, 0), 1, 1, fc=color) for color in LEGEND.values()],
    labels=LEGEND.keys(),
    loc="upper left",
    bbox_to_anchor=(-0.56, 1.3),
    ncol=1,
    fontsize=8,
)

ax.set_ylabel("Dependency Vector", fontsize=10, labelpad=10)
ax.set_xlabel("Dependency Surface", fontsize=10, labelpad=20)

CAPTION = {
    "P": "Partial Inline",
    "F": "Full Inline",
    "R": "Rename",
    "D": "Duplicate",
    r"$\Delta$": "Changed",
}

ax.text(
    -6.5,
    -2.5,
    "\n".join([f"{k}: {v}" for k, v in CAPTION.items()]),
    fontsize=8,
    ha="left",
    va="center",
)


save_fig(fig, "report", close=True)

[          mpl.py:84 ] INFO: Saved figure to /Users/szhong/Downloads/bpf-study/paper/figs/report.pdf
